In [2]:
import os
import pickle
import torch
import numpy as np
from math import ceil
from model_vc import Generator
import librosa
import librosa.display
import matplotlib.pyplot as plt

import math
import glob

baseline = pickle.load(open('/home/ytang363/7100_voiceConversion/results/other/002_baseline-results.pkl', "rb"))
retrain = pickle.load(open('/home/ytang363/7100_voiceConversion/results/other/002_results.pkl', "rb"))

In [3]:
# function: https://dsp.stackexchange.com/questions/56391/mel-cepstral-distortion
# paper: https://www.cs.cmu.edu/~awb/papers/sltu2008/kominek_black.sltu_2008.pdf
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

cost_function = log_spec_dB_dist

In [4]:
# Load ground truth utterance
rootDir = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel-16k'
dirName, subdirList, _ = next(os.walk(rootDir))
process_speakers = ['p225', 'p226', 'p227', 'p228', 'p229', 'p230', 'p231', 'p232', 'p233', 'p234', 'p237']
process_speakers = ['p231', 'p243', 'p272', 'p279', 'p314', 'p339'] # test spk list
process_uttr = '002'
gt = {}

def load_ground_truth(root_dir, process_speakers, process_uttr):
    gt = {}

    for speaker in sorted(os.listdir(root_dir)):
        if speaker not in process_speakers:
            continue

        # print('Processing speaker: %s' % speaker)
        speaker_dir = os.path.join(root_dir, speaker)
        _, _, file_list = next(os.walk(speaker_dir))

        indices = [i for i, element in enumerate(file_list) if process_uttr in element]
        if indices:
            print(file_list[indices[0]])

            spec = np.load(os.path.join(speaker_dir, file_list[indices[0]]))
            gt[speaker] = spec
        else:
            print(f"No matching file found for {process_uttr} in {speaker}")

    return gt

In [5]:
# Loop for evaluation
def avg_mcd(gen_mel, process_uttr):

    root_dir = '/home/ytang363/7100_voiceConversion/VCTK-Corpus-0.92/spmel-16k'
    process_speakers = ['p231', 'p243', 'p272', 'p279', 'p314', 'p339']
    gt = load_ground_truth(root_dir, process_speakers, process_uttr)

    total_diff = 0
    for i in range(len(process_speakers)):
        current = gen_mel[i][0].split('x')[1]
        test_uttr = gen_mel[i][1] 
        gt_uttr = gt[current]

        print(gt_uttr.shape, test_uttr.shape)
        print(gt_uttr.T.shape, test_uttr.T.shape)
        
        min_cost, _ = librosa.sequence.dtw(gt_uttr[:, 1:].T, test_uttr[:, 1:].T, metric=cost_function)
        # min_cost, _ = librosa.sequence.dtw(gt_uttr, test_uttr, metric=cost_function)  
        print(gt_uttr.shape)
        print(test_uttr.shape)
        print('-' * 10)                                                                            
        diff = np.mean(min_cost) / gt_uttr.shape[0]
        total_diff += diff
    avg_MCD = total_diff / len(process_speakers)
    return avg_MCD       

In [12]:
model = 'res_32'
uttr = '275'
epoch = '1500k'

path = f'/home/ytang363/7100_voiceConversion/results/result_test/{model}/{uttr}_{model}_{epoch}-results.pkl'
result = pickle.load(open(path, "rb"))
avg_mcd(result, uttr)

p231_275_mic1.npy
p243_275_mic1.npy
p272_275_mic1.npy
p279_275_mic1.npy
p314_275_mic1.npy
p339_275_mic1.npy
(185, 80) (185, 80)
(80, 185) (80, 185)
(185, 80)
(185, 80)
----------
(257, 80) (185, 80)
(80, 257) (80, 185)
(257, 80)
(185, 80)
----------
(131, 80) (185, 80)
(80, 131) (80, 185)
(131, 80)
(185, 80)
----------
(107, 80) (185, 80)
(80, 107) (80, 185)
(107, 80)
(185, 80)
----------
(190, 80) (185, 80)
(80, 190) (80, 185)
(190, 80)
(185, 80)
----------
(217, 80) (185, 80)
(80, 217) (80, 185)
(217, 80)
(185, 80)
----------


5.0766050918357974

In [7]:
print(np.__version__) # old numpy version 1.24.4

1.26.4


In [13]:
vec = [ 4.1677,
4.0212,
4.283,
4.4987,
5.0766,
3.8982
]
np.mean(vec)

4.324233333333333